## Archive

This notebook looks at a particular historical fire and generates all the output.

In [1]:
import datetime
import pandas as pd
import geopandas as gpd

from fireatlas import FireMain, FireTime, FireObj, FireConsts, postprocess, preprocess
from fireatlas.utils import timed

tst = [2020, 9, 5, "AM"]
ted = [2020, 9, 8, "PM"]
region = ("Creek", [-119.5, 36.8, -118.9, 37.7])

list_of_ts = list(FireTime.t_generator(tst, ted))

## Preprocess the region

Preprocess the region to get rid of static flare sources. Save that new "swiss cheese" shape off into a geojson file for later.

In [3]:
preprocess.preprocess_region(region)

2024-03-26 17:23:35,650 - FireLog - INFO - func:preprocess_region took: 626.01 ms


'data/FEDSpreprocessed/Creek/Creek.json'

## Are preprocessed input files available?

Check to see if all the files that we need have already been processed and are available on s3.

Note: "NOAA20" files for this time period are not in the input folder so we will only use "SNPP"

In [8]:
ts_that_need_preprocessing = preprocess.check_preprocessed_file(tst, ted, sat="SNPP", freq="monthly", location="s3")

In [12]:
assert len(ts_that_need_preprocessing) == 1, "There is one monthly file that is not preprocessed on s3"

Since there is one file that needs processing, let's process that and then we can read from local.

In [13]:
for t in ts_that_need_preprocessing:
    preprocess.preprocess_monthly_file(t, "SNPP")

2024-03-26 17:28:29,826 - FireLog - INFO - preprocessing VNP14IMGML.202009.C1.05.txt


Processing days: 100%|██████████| 30/30 [00:33<00:00,  1.11s/it]

2024-03-26 17:29:51,873 - FireLog - INFO - func:preprocess_input_file took: 1.37 min


## Preprocess for region and t

Do initial filtering and clustering using the preprocessed region (from local) and the half day files (also from local).

In [15]:
%%time
for t in list_of_ts:
    preprocess.preprocess_region_t(t, sensor="SNPP", region=region, read_location="local")

2024-03-26 17:30:40,299 - FireLog - INFO - func:read_region took: 1.41 ms
2024-03-26 17:30:40,299 - FireLog - INFO - filtering and clustering 2020-9-5 AM, SNPP, Creek
2024-03-26 17:30:40,330 - FireLog - INFO - func:read_preprocessed_input took: 30.33 ms
2024-03-26 17:30:46,881 - FireLog - INFO - func:do_clustering took: 383.37 ms
2024-03-26 17:30:46,895 - FireLog - INFO - func:preprocess_region_t took: 6.60 sec
2024-03-26 17:30:46,897 - FireLog - INFO - func:read_region took: 1.01 ms
2024-03-26 17:30:46,897 - FireLog - INFO - filtering and clustering 2020-9-5 PM, SNPP, Creek
2024-03-26 17:30:47,043 - FireLog - INFO - func:read_preprocessed_input took: 145.57 ms
2024-03-26 17:30:47,069 - FireLog - INFO - func:do_clustering took: 10.47 ms
2024-03-26 17:30:47,094 - FireLog - INFO - func:preprocess_region_t took: 198.05 ms
2024-03-26 17:30:47,095 - FireLog - INFO - func:read_region took: 1.02 ms
2024-03-26 17:30:47,096 - FireLog - INFO - filtering and clustering 2020-9-6 AM, SNPP, Creek
20

CPU times: user 1.75 s, sys: 167 ms, total: 1.92 s
Wall time: 8.24 s


## Run the fire expansion and merging algorithm

Now that we have all the region-specific preprocessed fires, we can go ahead and run `Fire_Forward`.

In [16]:
allfires, allpixels = FireMain.Fire_Forward(tst=tst, ted=ted, sat="SNPP", region=region, read_location="local")

2024-03-26 17:32:05,591 - FireLog - INFO - func:read_preprocessed took: 5.05 ms
2024-03-26 17:32:05,596 - FireLog - INFO - func:read_preprocessed took: 4.23 ms
2024-03-26 17:32:05,606 - FireLog - INFO - func:read_preprocessed took: 9.90 ms
2024-03-26 17:32:05,611 - FireLog - INFO - func:read_preprocessed took: 4.01 ms
2024-03-26 17:32:05,619 - FireLog - INFO - func:read_preprocessed took: 7.37 ms
2024-03-26 17:32:05,625 - FireLog - INFO - func:read_preprocessed took: 5.43 ms
2024-03-26 17:32:05,640 - FireLog - INFO - func:read_preprocessed took: 14.22 ms
2024-03-26 17:32:05,647 - FireLog - INFO - func:read_preprocessed took: 6.79 ms
2024-03-26 17:32:05,652 - FireLog - INFO - --------------------
2024-03-26 17:32:05,652 - FireLog - INFO - Fire tracking at [2020, 9, 5, 'AM']
2024-03-26 17:32:06,184 - FireLog - INFO - func:Fire_expand_rtree took: 529.83 ms
2024-03-26 17:32:06,186 - FireLog - INFO - func:Fire_merge_rtree took: 0.76 ms
2024-03-26 17:32:06,186 - FireLog - INFO - fids_expand:

We can save these files off in case something goes wrong later

In [17]:
postprocess.save_allpixels(allpixels, tst, ted, region)
postprocess.save_allfires_gdf(allfires.gdf, tst, ted, region)

2024-03-26 17:32:36,125 - FireLog - INFO - func:save_allpixels took: 154.77 ms
2024-03-26 17:32:36,177 - FireLog - INFO - func:save_allfires_gdf took: 51.20 ms


'data/FEDSoutput-s3-conus/Creek/2020/allfires_20200908_PM.parq'

## Write Snapshot files

Start by reading in the outputs from the run:

In [18]:
allpixels = postprocess.read_allpixels(tst, ted, region, location="local")
allfires_gdf = postprocess.read_allfires_gdf(tst, ted, region, location="local")

2024-03-26 17:32:37,247 - FireLog - INFO - func:read_allpixels took: 27.20 ms
2024-03-26 17:32:37,285 - FireLog - INFO - func:read_allfires_gdf took: 36.89 ms


Then iterate through all of the ts and save the snapshot layers.

In [19]:
%%time
postprocess.save_snapshots(allfires_gdf, region, tst, ted)

2024-03-26 17:32:48,300 - FireLog - INFO - func:save_snapshot_layers took: 431.14 ms
2024-03-26 17:32:48,437 - FireLog - INFO - func:save_snapshot_layers took: 133.32 ms
2024-03-26 17:32:48,606 - FireLog - INFO - func:save_snapshot_layers took: 165.06 ms
2024-03-26 17:32:48,768 - FireLog - INFO - func:save_snapshot_layers took: 159.86 ms
2024-03-26 17:32:48,921 - FireLog - INFO - func:save_snapshot_layers took: 149.10 ms
2024-03-26 17:32:49,055 - FireLog - INFO - func:save_snapshot_layers took: 131.61 ms
2024-03-26 17:32:49,230 - FireLog - INFO - func:save_snapshot_layers took: 172.76 ms
2024-03-26 17:32:49,373 - FireLog - INFO - func:save_snapshot_layers took: 138.92 ms
2024-03-26 17:32:49,374 - FireLog - INFO - func:save_snapshots took: 1.51 sec


CPU times: user 898 ms, sys: 18.3 ms, total: 916 ms
Wall time: 1.51 s


## Write Largefire files

First we will find all the large fires:

In [20]:
large_fires = postprocess.find_largefires(allfires_gdf)

2024-03-26 17:32:55,210 - FireLog - INFO - func:find_largefires took: 4.16 ms


In this case there is only one since we set the region and time specifically to look at th Creek Fire

In [21]:
large_fires

array([1])

First we'll use the `allpixels` object to create the `nplist` layer

In [23]:
%%time
postprocess.save_large_fires_nplist(allpixels, region, large_fires, tst)

2024-03-26 17:33:40,378 - FireLog - INFO - func:save_fire_nplist took: 698.67 ms
2024-03-26 17:33:40,379 - FireLog - INFO - func:save_large_fires_nplist took: 702.14 ms


CPU times: user 594 ms, sys: 110 µs, total: 594 ms
Wall time: 703 ms


The rest of the layers will be created directly from the `allfires_gdf` and have some merge handling

In [24]:
%%time
postprocess.save_large_fires_layers(allfires_gdf, region, large_fires, tst)

2024-03-26 17:33:41,190 - FireLog - INFO - func:merge_rows took: 14.44 ms


1 rows that potentially need a merge


2024-03-26 17:33:41,509 - FireLog - INFO - func:save_fire_layers took: 317.06 ms
2024-03-26 17:33:41,510 - FireLog - INFO - func:save_large_fires_layers took: 339.10 ms


CPU times: user 246 ms, sys: 42.8 ms, total: 289 ms
Wall time: 340 ms
